In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import strip_punctuation

from sklearn.decomposition import PCA

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import load_model

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(True)

Using TensorFlow backend.


## 01. Build your own embeddings & using pre-existing embeddings

### Build your own embeddings

In [2]:
sentences = [["this", "is", "the", "first", "sentence", "for", "word2vec"],
             ["this", "is", "the", "second", "sentence", "for", "word2vec"]]

In [3]:
# sg: 1 for skip-gram, 2 for CBOW
# alpha: learning rate
# window: windows size for skip-grams or CBOW
# min_count: any word counts that are less than this are ignored
# size: dimensions for each word
model = Word2Vec(sentences, alpha=0.025, window=5, min_count=1, sg=1, size=10)

In [4]:
print(model)

Word2Vec(vocab=8, size=10, alpha=0.025)


In [5]:
print(list(model.wv.vocab))

['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec', 'second']


In [6]:
model.wv[model.wv.vocab]

array([[ 0.0252312 ,  0.02013404,  0.0214923 ,  0.00263448,  0.01042301,
         0.03848768, -0.02364977, -0.01105659, -0.03465864,  0.01656245],
       [ 0.02091155, -0.02967838,  0.01292858, -0.03467149,  0.04431187,
        -0.01508507, -0.04757995, -0.03514276,  0.00977588,  0.0227128 ],
       [-0.01478777, -0.02572875,  0.02042815,  0.01532631,  0.02637147,
        -0.02083855,  0.00792016,  0.01655993,  0.0479192 ,  0.00823658],
       [ 0.01490779,  0.02584932, -0.03051792, -0.04860388, -0.01748068,
         0.04512254, -0.04027652,  0.04381217,  0.00112167, -0.00826722],
       [ 0.02168082,  0.0338375 , -0.03494537,  0.04093472, -0.00176067,
         0.00582241, -0.04751141,  0.01348697,  0.03560459, -0.00962873],
       [-0.03781706, -0.03867743, -0.03649498, -0.0134535 , -0.02848131,
        -0.01081789,  0.00266035,  0.04305999, -0.00649607, -0.00530395],
       [-0.04443134,  0.04345368,  0.04157907, -0.04900191,  0.02736149,
         0.01877012, -0.01018004, -0.03829909

In [7]:
model.save('model.bin')

In [8]:
model = Word2Vec.load('model.bin')

In [9]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(model.wv[model.wv.vocab])

In [10]:
pca.explained_variance_ratio_

array([0.39837837, 0.22384556, 0.17916888], dtype=float32)

In [11]:
pca_result_df = pd.DataFrame([pca_result[:,0], pca_result[:,1], pca_result[:,2]]).T
pca_result_df.columns = ["x", "y", "z"]
pca_result_df["word"] = list(model.wv.vocab)

In [12]:
pca_result_df

,x,y,z,word
0,0.042867,0.024495,-0.018991,this
1,0.038386,-0.043991,-0.034028,is
2,-0.026891,-0.064972,-0.007985,the
3,-0.012385,0.073182,0.011840,first
4,-0.042308,0.029931,-0.059509,sentence
5,-0.051368,-0.008007,0.063963,for
6,0.108087,0.000322,0.032475,word2vec
7,-0.056388,-0.010960,0.012236,second


In [13]:
trace1 = go.Scatter3d(x=pca_result_df["x"], y=pca_result_df["y"], z=pca_result_df["z"],
                      mode='markers', marker={"size": 4, "opacity": 0.8},
                      text=pca_result_df["word"])
data = [trace1]

layout = go.Layout(margin={"l": 0, "r": 0, "b": 0, "t": 0}, 
                   scene={"xaxis": {"title": "1st Principle Component"}, 
                          "yaxis": {"title": "2nd Principle Component"}, 
                          "zaxis": {"title": "3rd Principle Component"},
                          "annotations": [{"showarrow": False,
                                           "x": row["x"],
                                           "y": row["y"],
                                           "z": row["z"],
                                           "text": row["word"],
                                           "xanchor": "left",
                                           "xshift": 5,
                                           "opacity": 1} for _, row in pca_result_df.iterrows()]})

iplot(go.Figure(data=data, layout=layout))

### Use Stanford NLP Group's Glove Vectors

In [14]:
# 6B tokens, 400K vocab, uncased, 50d vectors
glove_vectors = KeyedVectors.load_word2vec_format('./vectors/word2vec.6B.50d.txt')

In [15]:
print(list(glove_vectors.vocab)[0:10])

['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]


In [16]:
glove_vectors[glove_vectors.vocab]

array([[ 0.418   ,  0.24968 , -0.41242 , ..., -0.18411 , -0.11514 ,
        -0.78581 ],
       [ 0.013441,  0.23682 , -0.16899 , ..., -0.56657 ,  0.044691,
         0.30392 ],
       [ 0.15164 ,  0.30177 , -0.16763 , ..., -0.35652 ,  0.016413,
         0.10216 ],
       ...,
       [-0.51181 ,  0.058706,  1.0913  , ..., -0.25003 , -1.125   ,
         1.5863  ],
       [-0.75898 , -0.47426 ,  0.4737  , ...,  0.78954 , -0.014116,
         0.6448  ],
       [ 0.072617, -0.51393 ,  0.4728  , ..., -0.18907 , -0.59021 ,
         0.55559 ]], dtype=float32)

In [17]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(glove_vectors[glove_vectors.vocab])

In [18]:
pca.explained_variance_ratio_

array([0.05418369, 0.04413847, 0.03602274])

In [19]:
pca_result_df = pd.DataFrame([pca_result[:,0], pca_result[:,1], pca_result[:,2]]).T
pca_result_df.columns = ["x", "y", "z"]
pca_result_df["word"] = list(glove_vectors.vocab)

In [20]:
pca_result_df = pca_result_df.head(300)

In [21]:
pca_result_df.head(10)

,x,y,z,word
0,5.217883,2.070946,-1.058571,the
1,4.080164,3.066622,-0.724225,","
2,4.607367,2.446008,-0.396058,.
3,5.051972,2.094044,-1.148352,of
4,5.047372,2.344891,-0.247560,to
5,4.548764,2.682500,-0.435750,and
6,5.171853,1.915757,-1.470239,in
7,5.013679,2.319080,-0.123463,a
8,3.894762,2.546774,-0.950930,""""
9,4.349796,3.510722,-0.871976,'s


In [22]:
trace1 = go.Scatter3d(x=pca_result_df["x"], y=pca_result_df["y"], z=pca_result_df["z"],
                      mode='markers', marker={"size": 4, "opacity": 0.8},
                      text=pca_result_df["word"])
data = [trace1]

layout = go.Layout(margin={"l": 0, "r": 0, "b": 0, "t": 0}, 
                   scene={"xaxis": {"title": "1st Principle Component"}, 
                          "yaxis": {"title": "2nd Principle Component"}, 
                          "zaxis": {"title": "3rd Principel Component"},
                          "annotations": [{"showarrow": False,
                                           "x": row["x"],
                                           "y": row["y"],
                                           "z": row["z"],
                                           "text": row["word"],
                                           "xanchor": "left",
                                           "xshift": 5,
                                           "opacity": 1} for _, row in pca_result_df.iterrows()]})

iplot(go.Figure(data=data, layout=layout))

In [23]:
pca_result_df = pd.DataFrame([pca_result[:,0], pca_result[:,1], pca_result[:,2]]).T
pca_result_df.columns = ["x", "y", "z"]
pca_result_df["word"] = list(glove_vectors.vocab)

In [24]:
pca_result_df = pca_result_df[pca_result_df["word"].isin(["king", "queen", "uncle", "aunt", "man", "woman"])]

In [25]:
pca_result_df

,x,y,z,word
300,4.167136,2.875631,-0.066294,man
691,3.352313,3.138729,-1.126750,king
787,3.939651,2.182287,-0.060381,woman
2060,3.366348,1.815066,-1.123616,queen
5152,1.839161,3.202459,-0.243355,uncle
9651,1.593296,2.387500,-0.397805,aunt


In [26]:
trace1 = go.Scatter3d(x=pca_result_df["x"], y=pca_result_df["y"], z=pca_result_df["z"],
                      mode='markers', marker={"size": 4, "opacity": 0.8},
                      text=pca_result_df["word"])
data = [trace1]

layout = go.Layout(margin={"l": 0, "r": 0, "b": 0, "t": 0}, 
                   scene={"xaxis": {"title": "1st Principle Component"}, 
                          "yaxis": {"title": "2nd Principle Component"}, 
                          "zaxis": {"title": "3rd Principel Component"},
                          "annotations": [{"showarrow": False,
                                           "x": row["x"],
                                           "y": row["y"],
                                           "z": row["z"],
                                           "text": row["word"],
                                           "xanchor": "left",
                                           "xshift": 5,
                                           "opacity": 1} for _, row in pca_result_df.iterrows()]})

iplot(go.Figure(data=data, layout=layout))

## 02. Sentiment Analysis our pre-built embeddings

In [27]:
documents = ["oh my god! So good. Philly grinder!",
             "This is one of the best pizza's in town. The 7 cheese pizza is A mazing.... stuffed churros, yum. Great upbeat staff. I could eat here every night.",
             "I'm telling you YOUR SERVICE STINKS AND YOU'D RETAIN MORE CUSTOMERS IF YOU'D JUST TEACH THEM TO USE THE PHONE PROPERLY AND TAKE AN ORDER PROPERLY.",
             "My favorite Fort Collins pizza place! I always go for the south of the border. They also have the best spicy ranch.",
             "Nope. Don't do it. Ordered a pizza over 2 hours ago and it still hasn't shown up.",
             "My favorite pizza place in fort collins! I always order South of the border with cream cheese the staff are awesome. One time the pizza deliver guy came super late. He apologized and gave us restaurant credit."]

labels = np.array([1, 1, 0, 1, 1, 0])

In [28]:
encoded_documents = []
for line in documents:
    line = strip_punctuation(line)
    encoded = []
    for word in line.split():
        try:
            encoded.append(glove_vectors.vocab[word].index)
        except:
            encoded.append(0)
    encoded_documents.append(encoded)

In [29]:
encoded_documents[0]

[3202, 192, 1533, 0, 219, 0, 39890]

In [30]:
max_length = max([len(i.split()) for i in documents])

In [31]:
max_length

37

In [32]:
padded_documents = pad_sequences(encoded_documents, maxlen=max_length, padding='post')

In [33]:
padded_documents[0]

array([ 3202,   192,  1533,     0,   219,     0, 39890,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0], dtype=int32)

In [34]:
embedding_input_dim = glove_vectors[glove_vectors.vocab].shape[0]
embedding_output_dim = glove_vectors[glove_vectors.vocab].shape[1]

In [35]:
embedding_input_dim

400000

In [36]:
embedding_output_dim

50

In [37]:
model = Sequential()
model.add(Embedding(input_dim=embedding_input_dim, output_dim=embedding_output_dim, weights=[glove_vectors[glove_vectors.vocab]], input_length=max_length))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer="adam")
model.fit(padded_documents, labels, epochs=50, verbose=0)
model.evaluate(padded_documents, labels, verbose=3)

0.00019652389164548367

In [38]:
model.save("model.h5")
model = load_model("model.h5")

In [39]:
model.predict(padded_documents)

array([[9.9983823e-01],
       [9.9999225e-01],
       [8.6409593e-04],
       [9.9996328e-01],
       [9.9989200e-01],
       [4.1845820e-07]], dtype=float32)

In [40]:
model.predict_classes(padded_documents)

array([[1],
       [1],
       [0],
       [1],
       [1],
       [0]], dtype=int32)

In [41]:
labels

array([1, 1, 0, 1, 1, 0])

In [42]:
accuracy_score(labels, model.predict_classes(padded_documents))

1.0

### Create an example document

In [43]:
documents = ["The pizza is great, I would highly recommend it",
             "The pizza is bad"]

In [44]:
encoded_documents = []
for line in documents:
    line = strip_punctuation(line)
    encoded = []
    for word in line.split():
        try:
            encoded.append(glove_vectors.vocab[word].index)
        except:
            encoded.append(0)
    encoded_documents.append(encoded)

In [45]:
encoded_documents[0]

[0, 9388, 14, 353, 0, 54, 1786, 7546, 20]

In [46]:
padded_documents = pad_sequences(encoded_documents, maxlen=max_length, padding='post')

In [47]:
padded_documents[0]

array([   0, 9388,   14,  353,    0,   54, 1786, 7546,   20,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [48]:
model.predict_classes(padded_documents)

array([[0],
       [0]], dtype=int32)

## Embedding Projector

https://projector.tensorflow.org/